In [1]:
import pandas as pd
import numpy as np

In [16]:
file_path = "/root/Software/Config/mysql_password.txt"
f = open(file_path, "r")
key_password_dict = {}
for x in f:
    key_password_pair = x.strip().split(":")
    key_password_dict[key_password_pair[0]] = key_password_pair[1]

f.close()

In [17]:
key_password_dict

{'user': 'rzdong', 'passwd': '13ProDrzusa!'}